In [ ]:
pip install -q google-generativeai

In [ ]:
import google.colab.userdata as userdata
import google.generativeai as genai

# Your Python tools as regular functions.
def say_hi(query: str) -> str:
    """
    Use this tool when the user greets you, says hello, or starts a conversation.
    """
    return "Hi!"

def say_bye(query: str) -> str:
    """
    Use this tool when the user wants to end the conversation or says goodbye.
    """
    return "Bye!"

def hi_follow_up(query: str) -> str:
    """
    Use this tool as follow-up responses when the user greets you, says hello, or starts a conversation.
    """
    return "How are you?"

def bye_follow_up(query: str) -> str:
    """
    Use this tool as follow-up responses when the user wants to end the conversation or says goodbye.
    """
    return "Have a great day!"

# A dictionary to map tool names to the actual functions.
tools_map = {
    "say_hi": say_hi,
    "say_bye": say_bye
}

follow_up_tools_map = {
    "hi_follow_up": hi_follow_up,
    "bye_follow_up": bye_follow_up,
}

In [ ]:
# Configure with your API key
api_key = userdata.get("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("Please set the GOOGLE_API_KEY environment variable.")
genai.configure(api_key=api_key)

# The SDK automatically uses Pydantic models to create the tool schema.
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[say_hi, say_bye]
)

follow_up_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[hi_follow_up,bye_follow_up]
)

In [ ]:
# System instruction (prompting) to guide the model's behavior.
system_instruction = ('''
You are an assistant designed to use tools to respond to user requests.
Your primary task is to carefully analyze the user's input and determine which of the available tools is most appropriate to accomplish their goal.

If a tool is a perfect match, call it with the correct parameters.
If no tool is suitable for the user's request, do not aexttempt to call any tool. Instead, respond directly with the exact phrase: 'You can do better!'
''')

system_instruction2 = ('''
You are a follow-up assistant agent.
Your job is to look at the name of the tool the previous agent called and, based on that name, select the appropriate follow-up tool to call with suggested followup response as a arguments.
For example, if the previous tool was say_hi, call hi_follow_up. If it was say_bye, call bye_follow_up.
''')

# The conversational loop
print("Agent is ready. Type 'exit' to end the conversation.")

for turn in range(10): # A for loop to limit the number of turns for this example

    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Agent: Goodbye!")
        break

   # Send the user's message to the model with the system instruction
    response = model.generate_content(
        [system_instruction, user_input]
        )

    # Check if the model's response contains a tool call
    try:
        tool_call = response.candidates[0].content.parts[0].function_call
        tool_name = tool_call.name
        tool_args = dict(tool_call.args)

        print(f"Agent 1 chose to call tool: {tool_name} with arguments: {tool_args}")

        # Execute the tool using the mapped function and the validated arguments

        tool_result = tools_map[tool_name](**tool_args)

        # Print the tool's output to the user
        print(f"Agent_1: {tool_result}")

        follow_up_response = follow_up_model.generate_content(
                                [system_instruction2, tool_name]
                             )

        try:
             follow_up_tool_call = follow_up_response.candidates[0].content.parts[0].function_call
             tool_name = follow_up_tool_call.name
             tool_args = dict(follow_up_tool_call.args)

             print(f"Agent 2 chose to call tool: {tool_name} with arguments: {tool_args}")

             # Execute the tool using the mapped function and the validated arguments
             tool_result = follow_up_tools_map[tool_name](**tool_args)

             # Print the tool's output to the user
             print(f"Agent_2: {tool_result}")
        except:
            print(f"Agent: {follow_up_response.text}")
    except :
        # This block runs if no tool was called. The model responded with text
        print(f"Agent: {response.text}")


Agent is ready. Type 'exit' to end the conversation.
You: hi
Agent 1 chose to call tool: say_hi with arguments: {'query': 'hi'}
Agent_1: Hi!
Agent 2 chose to call tool: hi_follow_up with arguments: {'query': 'Hello there! How can I help you today?'}
Agent_2: How are you?
You: bye
Agent 1 chose to call tool: say_bye with arguments: {'query': 'bye'}
Agent_1: Bye!
Agent 2 chose to call tool: bye_follow_up with arguments: {'query': ''}
Agent_2: Have a great day!
You: adios
Agent 1 chose to call tool: say_bye with arguments: {'query': 'adios'}
Agent_1: Bye!
Agent 2 chose to call tool: bye_follow_up with arguments: {'query': ''}
Agent_2: Have a great day!
You: namaste
Agent 1 chose to call tool: say_hi with arguments: {'query': 'namaste'}
Agent_1: Hi!
Agent 2 chose to call tool: hi_follow_up with arguments: {'query': 'Hello!'}
Agent_2: How are you?
You: merhaba
Agent 1 chose to call tool: say_hi with arguments: {'query': 'merhaba'}
Agent_1: Hi!
Agent 2 chose to call tool: hi_follow_up with a